## Import

In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [162]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

In [163]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

## Data Load

In [164]:
train_df = pd.read_csv('/content/train.csv')

In [147]:
train_df.head()

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [165]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

# 데이터 탐색

In [16]:
# 열 이름. 데이터 형식, 값 개수 확인
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39607 entries, 0 to 39606
Data columns (total 71 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      39607 non-null  object 
 1   X_01    39607 non-null  float64
 2   X_02    39607 non-null  float64
 3   X_03    39607 non-null  float64
 4   X_04    39607 non-null  int64  
 5   X_05    39607 non-null  float64
 6   X_06    39607 non-null  float64
 7   X_07    39607 non-null  float64
 8   X_08    39607 non-null  float64
 9   X_09    39607 non-null  float64
 10  X_10    39607 non-null  float64
 11  X_11    39607 non-null  float64
 12  X_12    39607 non-null  float64
 13  X_13    39607 non-null  float64
 14  X_14    39607 non-null  float64
 15  X_15    39607 non-null  float64
 16  X_16    39607 non-null  float64
 17  X_17    39607 non-null  float64
 18  X_18    39607 non-null  float64
 19  X_19    39607 non-null  float64
 20  X_20    39607 non-null  float64
 21  X_21    39607 non-null  float64
 22

In [ ]:
# 기초 통계량 
train_df.describe()

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
count,39607.000000,39607.000000,39607.000000,39607.0,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,...,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000
mean,68.412040,103.320166,68.826354,1.0,102.337203,70.597211,29.407490,164.449320,225.397470,0.002449,...,31.290467,16.529382,3.155054,-26.294839,-26.308623,-22.400062,24.325061,-26.237762,-26.233869,-26.245868
std,2.655983,0.000372,5.151167,0.0,0.548353,2.259820,7.338204,220.402444,66.734725,0.086255,...,2.543222,1.893014,0.418940,0.660537,0.653580,0.920952,0.830197,0.656329,0.655090,0.655989
min,56.268000,103.320000,56.470000,1.0,101.774000,61.726000,14.140000,38.460000,37.580000,0.000000,...,18.589000,-19.963000,0.502000,-29.652000,-29.523000,-31.119000,19.844000,-29.544000,-29.448000,-29.620000
25%,66.465000,103.320000,65.070000,1.0,101.949000,68.864000,27.890000,105.990000,188.540000,0.000000,...,29.768000,16.146000,2.863000,-26.689000,-26.702000,-22.871000,23.836000,-26.630000,-26.624000,-26.640000
50%,68.504000,103.320000,67.270000,1.0,102.006000,69.884000,28.840000,115.040000,234.450000,0.000000,...,31.710000,16.694000,3.126000,-26.254000,-26.266000,-22.275000,24.420000,-26.198000,-26.193000,-26.204000
75%,69.524000,103.320000,71.770000,1.0,103.144000,71.923000,29.870000,132.620000,263.960000,0.000000,...,33.184000,17.164000,3.433500,-25.855000,-25.871000,-21.791000,24.911500,-25.799000,-25.794000,-25.809000
max,84.820000,103.321000,89.170000,1.0,103.160000,87.219000,163.860000,2387.440000,637.490000,3.600000,...,37.250000,18.998000,5.299000,-23.785000,-23.960000,-20.052000,26.703000,-23.722000,-23.899000,-23.856000


In [ ]:
# 결측치 확인
print(train_df.isna().sum())

ID      0
X_01    0
X_02    0
X_03    0
X_04    0
       ..
Y_10    0
Y_11    0
Y_12    0
Y_13    0
Y_14    0
Length: 71, dtype: int64


In [ ]:
# 상관관계 확인
print(df_corr)

          X_01      X_02      X_03  X_04      X_05      X_06      X_07  \
X_01  1.000000 -0.084611  0.211988   NaN  0.553974  0.626521 -0.008373   
X_02 -0.084611  1.000000 -0.028076   NaN -0.048915 -0.047862 -0.003771   
X_03  0.211988 -0.028076  1.000000   NaN -0.076944  0.407266  0.011814   
X_04       NaN       NaN       NaN   NaN       NaN       NaN       NaN   
X_05  0.553974 -0.048915 -0.076944   NaN  1.000000  0.104653 -0.032107   
...        ...       ...       ...   ...       ...       ...       ...   
Y_10  0.034081 -0.008553  0.107851   NaN  0.001554  0.060159 -0.017691   
Y_11  0.009947 -0.007557  0.034114   NaN  0.013407  0.012831 -0.002733   
Y_12  0.066904 -0.013885  0.111755   NaN  0.033491  0.070004 -0.015829   
Y_13  0.068074 -0.013285  0.112385   NaN  0.034294  0.069610 -0.016125   
Y_14  0.067083 -0.013482  0.109578   NaN  0.034546  0.069117 -0.017208   

          X_08      X_09      X_10  ...      Y_05      Y_06      Y_07  \
X_01  0.007622 -0.033579  0.011021  ..

## 전처리
###### 1. X4 : 1차 검사 통과 여부 // 범주로 처리 후 가변수화
###### 2. X23 : 2차 검사 통과 여부 // 범주로 처리 후 가변수화
###### 3. X50 ~ 56 // 이상치 평균치로 처리 or 선형보간법

In [61]:
# X4 : 1차 검사 통과 여부 // 범주로 처리
train_x['X_04'] = train_x['X_04'].map({1:'통과', 0:'탈락'})

# 확인
print(train_x.head())

     X_01     X_02   X_03 X_04     X_05    X_06   X_07    X_08    X_09  X_10  \
0  70.544  103.320  67.47   통과  101.892  74.983  29.45   62.38  245.71   0.0   
1  69.524  103.321  65.17   통과  101.944  72.943  28.73   61.23  233.61   0.0   
2  72.583  103.320  64.07   통과  103.153  72.943  28.81  105.77  272.20   0.0   
3  71.563  103.320  67.57   통과  101.971  77.022  28.92  115.21  255.36   0.0   
4  69.524  103.320  63.57   통과  101.981  70.904  29.68  103.38  241.46   0.0   

   ...  X_47  X_48      X_49        X_50        X_51        X_52        X_53  \
0  ...     1     1   9706.03  137.043591  135.359219  147.837968  134.313475   
1  ...     1     1  10423.43  133.736691  135.979817  149.924692  123.630583   
2  ...     1     1  10948.53  132.805112  131.055355  146.814592  128.939070   
3  ...     1     1  15007.03  134.138760  133.239422  139.720132  132.260824   
4  ...     1     1  11051.03  142.728970  136.620022  134.853555  134.760252   

         X_54        X_55        X_56 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [62]:
# X23 : 2차 검사 통과 여부 // 범주로 처리
train_x['X_23'] = train_x['X_23'].map({1:'통과', 0:'탈락'})

# 확인
print(train_x.head())

     X_01     X_02   X_03 X_04     X_05    X_06   X_07    X_08    X_09  X_10  \
0  70.544  103.320  67.47   통과  101.892  74.983  29.45   62.38  245.71   0.0   
1  69.524  103.321  65.17   통과  101.944  72.943  28.73   61.23  233.61   0.0   
2  72.583  103.320  64.07   통과  103.153  72.943  28.81  105.77  272.20   0.0   
3  71.563  103.320  67.57   통과  101.971  77.022  28.92  115.21  255.36   0.0   
4  69.524  103.320  63.57   통과  101.981  70.904  29.68  103.38  241.46   0.0   

   ...  X_47  X_48      X_49        X_50        X_51        X_52        X_53  \
0  ...     1     1   9706.03  137.043591  135.359219  147.837968  134.313475   
1  ...     1     1  10423.43  133.736691  135.979817  149.924692  123.630583   
2  ...     1     1  10948.53  132.805112  131.055355  146.814592  128.939070   
3  ...     1     1  15007.03  134.138760  133.239422  139.720132  132.260824   
4  ...     1     1  11051.03  142.728970  136.620022  134.853555  134.760252   

         X_54        X_55        X_56 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
# 가변수화
dummy_vars = ['X_23', 'X_04']
train_x = pd.get_dummies(train_x, columns=dummy_vars, drop_first=True)

# 확인
train_x

,X_01,X_02,X_03,X_05,X_06,X_07,X_08,X_09,X_10,X_11,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,70.544,103.320,67.47,101.892,74.983,29.45,62.38,245.71,0.0,0.0,...,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,101.944,72.943,28.73,61.23,233.61,0.0,0.0,...,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,103.153,72.943,28.81,105.77,272.20,0.0,0.0,...,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,101.971,77.022,28.92,115.21,255.36,0.0,0.0,...,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,101.981,70.904,29.68,103.38,241.46,0.0,0.0,...,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11588,67.485,103.320,78.87,101.957,69.884,29.64,72.94,255.21,0.0,0.0,...,1,1,11520.03,134.758504,137.312390,144.596620,133.276419,137.082584,140.914288,134.595951
11589,68.504,103.320,79.77,101.944,70.904,29.50,114.34,272.26,0.0,0.0,...,1,1,13195.53,134.135060,134.084187,136.621614,123.127989,128.433346,137.831639,131.160282
11590,66.465,103.320,75.57,103.152,66.825,29.64,98.75,230.12,0.0,0.0,...,1,1,13477.03,131.705205,138.880573,148.926584,125.926962,120.038624,140.762806,129.161236
11591,67.485,103.320,79.77,101.985,70.904,29.84,111.67,265.97,0.0,0.0,...,1,1,13807.13,139.000211,141.343060,138.841959,134.761427,135.982128,137.174321,133.236202


In [149]:
# X50 ~ 56 // 이상치 평균치로 처리

train_x['X_50'].replace('######',np.NaN)
train_x['X_51'].replace('######',np.NaN)
train_x['X_52'].replace('######',np.NaN)
train_x['X_53'].replace('######',np.NaN)
train_x['X_54'].replace('######',np.NaN)
train_x['X_55'].replace('######',np.NaN)
train_x['X_56'].replace('######',np.NaN)

mean_X10 = train_x['X_50'].mean()
mean_X11 = train_x['X_51'].mean()
mean_X11 = train_x['X_52'].mean()
mean_X11 = train_x['X_53'].mean()
mean_X11 = train_x['X_54'].mean()
mean_X11 = train_x['X_55'].mean()
mean_X11 = train_x['X_56'].mean()

# 채우기
train_x['X_50'].fillna(mean_X10, inplace=True)
train_x['X_51'].fillna(mean_X11, inplace=True)
train_x['X_52'].fillna(mean_X11, inplace=True)
train_x['X_53'].fillna(mean_X11, inplace=True)
train_x['X_54'].fillna(mean_X11, inplace=True)
train_x['X_55'].fillna(mean_X11, inplace=True)
train_x['X_56'].fillna(mean_X11, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
# 확인
train_x['X_50']

0        137.043591
1        133.736691
2        132.805112
3        134.138760
4        142.728970
            ...    
39602    129.965741
39603    127.633885
39604    132.501286
39605    128.189679
39606    135.096272
Name: X_50, Length: 39607, dtype: float64

In [166]:
# X50 ~ 56 // 이상치 선형보간법으로 처리

train_x['X_50'].replace('######',np.NaN)
train_x['X_51'].replace('######',np.NaN)
train_x['X_52'].replace('######',np.NaN)
train_x['X_53'].replace('######',np.NaN)
train_x['X_54'].replace('######',np.NaN)
train_x['X_55'].replace('######',np.NaN)
train_x['X_56'].replace('######',np.NaN)

# 채우기
train_x['X_50'].interpolate(method='linear', inplace=True)
train_x['X_51'].interpolate(method='linear', inplace=True)
train_x['X_52'].interpolate(method='linear', inplace=True)
train_x['X_53'].interpolate(method='linear', inplace=True)
train_x['X_54'].interpolate(method='linear', inplace=True)
train_x['X_55'].interpolate(method='linear', inplace=True)
train_x['X_56'].interpolate(method='linear', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


## XGBoost Model Fit

In [80]:
train_x.shape

(39607, 56)

In [81]:
train_y.shape

(39607, 14)

In [100]:
!pip install pycaret[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 7.0 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 134 kB 57.2 MB/s 
     |████████████████████████████████| 17.0 MB 41.5 MB/s 
     |████████████████████████████████| 167 kB 54.7 MB/s 
     |████████████████████████████████| 2.0 MB 50.9 MB/s 
     |████████████████████████████████| 10.4 MB 31.3 MB/s 
     |████████████████████████████████| 1.3 MB 71.1 MB/s 
     |████████████████████████████████| 262 kB 61.3 MB/s 
     |████████████████████████████████| 88 kB 7.9 MB/s 
     |████████████████████████████████| 6.8 MB 55.7 MB/s 
     |████████████████████████████████| 3.3 MB 36.2 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 1.7 MB 51.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  In

In [167]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

In [168]:
def mean_norm(df_input):# 평균정규화로 Pandas 데이터 프레임 정규화
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
def minmax_norm(df_input):#최소-최대정규화로 Pandas 데이터 프레임 정규화
    return (df_input - df_input.min()) / ( df_input.max() - df_input.min())
def quantile_norm(df_input):#quantile 정규화로 Pandas 데이터 프레임 정규화
    sorted_df = pd.DataFrame(np.sort(df_input.values,axis=0), index=df_input.index, columns=df_input.columns)
    mean_df = sorted_df.mean(axis=1)
    mean_df.index = np.arange(1, len(mean_df) + 1)
    quantile_df =df_input.rank(method="min").stack().astype(int).map(mean_df).unstack()
    return(quantile_df)

In [169]:
from sklearn import metrics 
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [170]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_df, train_size=0.8)

In [157]:
train_x = mean_norm(train_df.filter(regex='X')).dropna(axis=1) # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = mean_norm(valid_df.filter(regex='X')).dropna(axis=1) # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

In [158]:
import xgboost as xgb
XGB = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 0, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)

print("----------- 모델",XGB,'------------스코어  ',score)

[14:11:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:12:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:13:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:13:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [159]:
train_x = minmax_norm(train_df.filter(regex='X')).dropna(axis=1) # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = minmax_norm(valid_df.filter(regex='X')).dropna(axis=1) # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

In [160]:
import xgboost as xgb
XGB = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 0, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)

print("----------- 모델",XGB,'------------스코어  ',score)

[14:16:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:16:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:16:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
# 선형 보간법 처리 후 

In [171]:
train_x = minmax_norm(train_df.filter(regex='X')).dropna(axis=1) # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

valid_x = minmax_norm(valid_df.filter(regex='X')).dropna(axis=1) # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature

In [172]:
import xgboost as xgb
XGB = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 0, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)

print("----------- 모델",XGB,'------------스코어  ',score)

[14:22:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:24:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [88]:
# predicting
np.mean((multioutputregressor.predict(train_x) - train_y)**2, axis=0)

Y_01    0.116907
Y_02    0.138529
Y_03    0.121904
Y_04    6.579652
Y_05    6.076320
Y_06    2.881057
Y_07    0.163703
Y_08    0.390205
Y_09    0.384249
Y_10    0.736238
Y_11    0.647439
Y_12    0.386686
Y_13    0.384588
Y_14    0.386423
dtype: float64

## Inference

## Submit

In [ ]:
test_x = pd.read_csv('./test.csv').drop(columns=['ID'])

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 해커톤 data/open/sample_submission.csv')

In [ ]:
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')

Done.


In [ ]:
submit.to_csv('./submit.csv', index=False)